In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests

class Scraper:
    def __init__(self, poet_names=None, url=None):
        self.poet_names = poet_names
        self.url = url
        
    
    def set_poets_name(self, url: str)-> list:
        if url == "https://ganjoor.net/":
            self.url = url
            html_text = requests.get(url).text
            html_soup = BeautifulSoup(html_text,"lxml")
            divs_list = html_soup.find_all("div", class_="caption")
            addresses = [str(div.find("a")) for div in divs_list]
            poet_names = [address.split('"')[1][1:] for address in addresses]
            poet_names = list(set(poet_names))
            
            self.poet_names = poet_names
        else:
            raise ValueError("This program is not defined for scraping the provided URL.")
            
    @staticmethod
    def check_url(url: str)-> bool:
        try:
            get = requests.get(url)
            # if the request succeeds 
            if get.status_code == 200:
                return True
            else:
                return False
        #Exception
        except:
            return False
    
    def extend_url(self, extention):
        return self.url + extention

In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("./src")
ganjoor = Scraper()
ganjoor.set_poets_name("https://ganjoor.net/")

# New Section

In [ ]:
with open("ganjoor_poet_names.txt", "w") as f:
    for name in ganjoor.poet_names[:-1]:
        f.write(f"{name}\n")
    f.write(ganjoor.poet_names[-1])

In [ ]:
poet_names = []
with open("ganjoor_poet_names.txt","r") as f:
    for name in f.read().splitlines():
        poet_names.append(name)

In [ ]:
poet_names.__len__()

In [ ]:
def make_poets_url(names, cls):
    poets_url = [cls.extend_url(name) for name in names]
    return poets_url

In [ ]:
poets_url = make_poets_url(ganjoor.poet_names, ganjoor)

In [ ]:
import requests
from bs4 import BeautifulSoup
def get_first_layer(url):
    html_text = requests.get(url).text
    html_soup = BeautifulSoup(html_text,"lxml")

    divs_list = html_soup.find_all("div", class_="part-title-block")
    addresses = [str(div.find("a")) for div in divs_list]
    first_layer = [layer.split('"')[1].split("/")[-1] for layer in addresses]
    return first_layer

In [ ]:
import requests
from bs4 import BeautifulSoup
def get_second_layer(url):
    html_text = requests.get(url).text
    html_soup = BeautifulSoup(html_text,"lxml")

    divs_list = html_soup.find_all("div", class_="part-title-block")
    addresses = [str(div.find("a")) for div in divs_list]
    second_layer = [layer.split('"')[1].split("/")[-1] for layer in addresses if "#index" not in layer]
    if second_layer:
        return second_layer
    else:
        return ["",]

In [ ]:
poet_layers = dict()
poet_names_urls = list(zip(poet_names,poets_url))

for name, initial_layer in poet_names_urls:
    poet_layers[name] = list()
    
    for first_layer in get_first_layer(initial_layer):
        for second_layer in get_second_layer(f"{initial_layer}/{first_layer}"):
            if not second_layer:
                second_layer = ""
                slash = ""
            else:
                slash = "/"
            
            poet_layers[name].append(f"{initial_layer}/{first_layer}{slash}{second_layer}")
    print(name)
    


In [ ]:
f = open("second_layer_links", "w")
f.close()
poet_layers["rashhe"] = ["https://ganjoor.net/rashhe"]
for name, urls in poet_layers.items():
    if not urls:
        print(name)
    with open("second_layer_links.txt", "a+") as f:
        for url in urls:
            f.write(f"{url}\n")

In [ ]:
third_layer_links = []
with open("second_layer_links.txt","r") as f:
    for link in f.read().splitlines():
        third_layer_links.append(link)

In [ ]:
third_layer_links.__len__()

In [ ]:
all_links = []
for link in third_layer_links:
  html_text = requests.get(link).text
  html_soup = BeautifulSoup(html_text,"lxml")
  par_list = html_soup.find_all("p", class_="poem-excerpt")
  addresses = [str(div.find("a")) for div in par_list]
  prefixs = [address.split('"')[1].split("/")[-1] for address in addresses]
  for prefix in prefixs:
    all_links.append(link+"/"+prefix)
with open("final_list.txt", "w") as f:
    for name in all_links[:-1]:
        f.write(f"{name}\n")
    f.write(all_links[-1])

In [ ]:
len(all_links)